In [1]:
import cv2
import numpy as np
des_pts = np.array([[200,100], [800,100], [200,700], [800,700]])
des_cm = np.array([[200,100], [383,100], [200,283], [383,283]])
src_pts = np.array([[516,22], [971,182], [206,392], [730,685]])

h, status = cv2.findHomography(src_pts, des_cm)
np.save('h_cm.npy', h, False)

### live perspective transform

In [3]:
import cv2, time
import numpy as np
vid = cv2.VideoCapture(-1)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(2)
h = np.load('h_from_board.npy')
while True:
    _, img = vid.read()
    img = cv2.circle(img, (500, 500), 1, color=(0,0,255), thickness=9)
    warp_img = cv2.warpPerspective(img, h, (img.shape[1],img.shape[0]))
    cv2.imshow('img', warp_img)
   #  cv2.imwrite('template.jpg', img)
   #  if True:
   #     break
    if cv2.waitKey(1) & 0xFF == ord('q'):
      break
cv2.waitKey(0)
cv2.destroyAllWindows()
vid.release()

### LIVE FEED WITH DOTS

In [6]:
import cv2, time
import mediapipe as mp
import numpy as np
import time


mp_objectron = mp.solutions.objectron
vid = cv2.VideoCapture(0)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(2)


iterations = [1, 2, 5, 6]
h = np.load('h.npy')
temp_img = cv2.imread('template.jpg')
while True:
    start_time = time.time()
    _, img = vid.read()
    with mp_objectron.Objectron(static_image_mode=True, max_num_objects=6, min_detection_confidence=0.5, model_name='Shoe') as objectron:
   
      results = objectron.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

      if not results.detected_objects:
        print('No box landmarks detected on shoe', end= '\r')
        continue
      #print('Box landmarks of shoe:')
      ann_img = img.copy()
      temp_img_copy = temp_img.copy()
      display_img = temp_img_copy
      for detected_object in results.detected_objects:
        for i in iterations:
          x = int(detected_object.landmarks_2d.landmark[i].x * img.shape[1])
          y = int(detected_object.landmarks_2d.landmark[i].y * img.shape[0])
          display_img = cv2.circle(display_img, (x,y), 1, color=(0, 0, 255), thickness=5)
      if cv2.waitKey(1) & 0xFF == ord('q'):
        break
      warp_img = cv2.warpPerspective(display_img, h, (img.shape[1],img.shape[0]))
      cv2.imshow('img', warp_img)
    end_time = time.time()
    print(end_time - start_time)
cv2.waitKey(0)
cv2.destroyAllWindows()
vid.release()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


0.23570489883422852tected on shoe
0.11082768440246582
0.10860681533813477
0.13283014297485352
0.11230254173278809
0.13887333869934082
0.12182855606079102
0.12743306159973145
0.14894795417785645
0.14778685569763184
0.13759231567382812
0.13811373710632324
0.1384279727935791
0.16225862503051758
0.14025115966796875
0.12518072128295898
0.16005492210388184
0.14013433456420898
0.15975189208984375
0.1237020492553711
0.1411583423614502
0.14600491523742676
0.11991691589355469
0.14040136337280273
0.16180419921875
0.15850830078125
0.147810697555542
0.14742684364318848
0.1436600685119629
0.1376664638519287
0.14150571823120117
0.13904142379760742
0.15182161331176758
0.14049911499023438
0.14096307754516602
0.12862157821655273
0.14337611198425293
0.12260007858276367
0.12872099876403809
0.15378952026367188
0.1653439998626709
0.1458444595336914
0.13367438316345215
0.14350366592407227
0.11677980422973633
0.1110074520111084
0.14565634727478027
0.12224817276000977
0.12016415596008301
0.13680481910705566
0.

###  LIVE FEED WITH CIRCLE AND DIRECTION - SINGLE PERSON ONLY

In [1]:
import cv2
import numpy as np
import time

import mediapipe as mp

#img = cv2.imread('shoe.png')
#img = cv2.imread('/home/hameed/Downloads/shoes.jpeg')
vid = cv2.VideoCapture(-1)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(2)

def midpoint(x1, y1, x2, y2):
    return ((x1 + x2)/2, (y1 + y2)/2)

def point_transform(p, h):
    p = np.array(p, dtype='float32').reshape((1,1,2))
    p_out = cv2.perspectiveTransform(p, h)[0,0,:]
    return p_out


mp_objectron = mp.solutions.objectron

fx, fy = (1.0, 1.0)
px, py = (0.0, 0.0)
iterations = [1, 2, 5, 6]
h = np.load('h_from_board.npy')


axis_world = np.float32([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])

t_end = time.time() + 60 * 15
while time.time() < t_end:
    _, img = vid.read()
    objectron = mp_objectron.Objectron(mp_objectron.Objectron(static_image_mode=True, max_num_objects=5, min_detection_confidence=0.5, model_name='Shoe'))
    results = objectron.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ann_img = img.copy()
    #warp_img = img.copy()
    x = []
    y = []
    z_axis = []
    if not results.detected_objects:
        print('no detected objects', end='\r')
        continue
    for obj in results.detected_objects:
        
        x.append(int(obj.landmarks_2d.landmark[0].x * img.shape[1]))
        y.append(int(obj.landmarks_2d.landmark[0].y * img.shape[0]))

        axis_cam = np.matmul(obj.rotation, 0.1 * axis_world.T).T + obj.translation
        x_ori = axis_cam[..., 0]
        y_ori = axis_cam[..., 1]
        z_ori = axis_cam[..., 2]
    # Project 3D points to NDC space.
        x_ndc = np.clip(-fx * x_ori / (z_ori + 1e-5) + px, -1., 1.)
        y_ndc = np.clip(-fy * y_ori / (z_ori + 1e-5) + py, -1., 1.)
    # Convert from NDC space to image space.
        x_im = np.int32((1 + x_ndc) * 0.5 * img.shape[1])
        y_im = np.int32((1 - y_ndc) * 0.5 * img.shape[0])
        z_axis.append((x_im[3], y_im[3]))

    try:
        x, y = midpoint(x[0], y[0], x[1], y[1])
        x, y = point_transform((x,y), h)
        z0, z1 = midpoint(z_axis[0][0], z_axis[0][1], z_axis[1][0], z_axis[1][1])
        z0, z1 = point_transform((z0, z1), h)

        warp_img = cv2.warpPerspective(ann_img, h, (img.shape[1],img.shape[0]))
        warp_img = cv2.circle(warp_img, (int(x), int(y)), radius=100, color=(0, 0, 255), thickness=5)
        warp_img = cv2.arrowedLine(warp_img, (int(x), int(y)), (int(z0), int(z1)), color=(0, 128, 0), thickness=3, tipLength=0.5)
    except:
        print('not enough points', end='\r')

    try:
        cv2.imshow('ann_img', warp_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    except:
        print('wait', end='\r')
cv2.waitKey(0)
cv2.destroyAllWindows()
vid.release()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


###  LIVE FEED WITH CIRCLE AND DIRECTION - 2 PERSONS

In [9]:
import cv2
import numpy as np
import time
from itertools import combinations
import mediapipe as mp
import math

#img = cv2.imread('shoe.png')
#img = cv2.imread('/home/hameed/Downloads/shoes.jpeg')
vid = cv2.VideoCapture(-1)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(2)


def point_transform(p, h):
    p = np.array(p, dtype='float32').reshape((1,1,2))
    p_out = cv2.perspectiveTransform(p, h)[0,0,:]
    return p_out

def midpoint(center, pair):
    x1 = center[pair[0]][0]
    x2 = center[pair[1]][0]
    y1 = center[pair[0]][1]
    y2 = center[pair[0]][1]
    return ((x1 + x2)/2, (y1 + y2)/2)

def fetch_value(obj_index):
    return {
        0 : (0,1),
        1 : (0,2),
        2 : (0,3),
        3 : (1,2),
        4 : (1,3),
        5 : (2,3)
    }[obj_index]

mp_objectron = mp.solutions.objectron

fx, fy = (1.0, 1.0)
px, py = (0.0, 0.0)
iterations = [1, 2, 5, 6]
h = np.load('h_from_board.npy')


axis_world = np.float32([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])

t_end = time.time() + 60 * 15
while time.time() < t_end:
    _, img = vid.read()
    objectron = mp_objectron.Objectron(mp_objectron.Objectron(static_image_mode=True, max_num_objects=5, min_detection_confidence=0.5, model_name='Shoe'))
    results = objectron.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ann_img = img.copy()
    #warp_img = img.copy()
    center = []
    z_axis = []
    if not results.detected_objects:
        print('no detected objects', end='\r')
        continue
    for obj in results.detected_objects:
        
        x = (int(obj.landmarks_2d.landmark[0].x * img.shape[1]))
        y = (int(obj.landmarks_2d.landmark[0].y * img.shape[0]))
        center.append((x,y))

        axis_cam = np.matmul(obj.rotation, 0.1 * axis_world.T).T + obj.translation
        x_ori = axis_cam[..., 0]
        y_ori = axis_cam[..., 1]
        z_ori = axis_cam[..., 2]
    # Project 3D points to NDC space.
        x_ndc = np.clip(-fx * x_ori / (z_ori + 1e-5) + px, -1., 1.)
        y_ndc = np.clip(-fy * y_ori / (z_ori + 1e-5) + py, -1., 1.)
    # Convert from NDC space to image space.
        x_im = np.int32((1 + x_ndc) * 0.5 * img.shape[1])
        y_im = np.int32((1 - y_ndc) * 0.5 * img.shape[0])
        z_axis.append((x_im[3], y_im[3]))

    comb = combinations(center, 2)
    comb = list(comb)
    dist = []
    try:
        for i in range(len(comb)):

            p1, p2 = comb[i]
            dist.append(math.dist(p1, p2))

        sorted = dist.copy()
        sorted.sort()
        obj_index = dist.index(sorted[0]), dist.index(sorted[1])
        pairs = fetch_value(obj_index[0]), fetch_value(obj_index[1])
        #second_pair = fetch_value(obj_index[1])
        for pair in pairs:
            x, y = midpoint(center, pair)
            z0, z1 = midpoint(z_axis, pair)
            cv2.circle(ann_img, (int(x), int(y)), radius=60, color=(0, 0, 255), thickness=5)
            cv2.arrowedLine(ann_img, (int(x), int(y)), (int(z0), int(z1)), color=(0, 128, 0), thickness=3, tipLength=3)
        cv2.imshow('img', ann_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    
    except:
        print('not enough points', end='\r')

cv2.waitKey(0)
cv2.destroyAllWindows()
vid.release()

not enough pointsts
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough points
not enough pointsts
not enough points
not enough points
not enough points
not enough points
not enough points
not enough pointsts
not enough points
not enough points
not enough points
not enough points


In [7]:
center


[(788, 280), (693, 281)]

In [48]:
def point_transform(p, h):
    p = np.array(p, dtype='float32').reshape((1,1,2))
    p_out = cv2.perspectiveTransform(p, h)[0,0,:]
    return p_out

In [57]:
a, b = point_transform((400,400), h)
a, b


(387.8386, 613.14435)

### Live feed, multiple people

In [ ]:
import cv2
import numpy as np
import time
from itertools import combinations
import mediapipe as mp
import math

#img = cv2.imread('shoe.png')
#img = cv2.imread('/home/hameed/Downloads/shoes.jpeg')
vid = cv2.VideoCapture(-1)
vid.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
vid.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
time.sleep(2)


def point_transform(p, h):
    p = np.array(p, dtype='float32').reshape((1,1,2))
    p_out = cv2.perspectiveTransform(p, h)[0,0,:]
    return p_out

def midpoint(center, pair):
    x1 = center[pair[0]][0]
    x2 = center[pair[1]][0]
    y1 = center[pair[0]][1]
    y2 = center[pair[0]][1]
    return ((x1 + x2)/2, (y1 + y2)/2)

def fetch_value(obj_index):
    return {
        0 : (0,1),
        1 : (0,2),
        2 : (0,3),
        3 : (0,4),
        4 : (0,5),
        5 : (1,2),
        6 : (1,3),
        7 : (1,4),
        8 : (1,5),
        9 : (2,3),
        10 : (2,4),
        11: (2,5),
        12 : (3,4),
        13 : (3,5),
        14 : (4,5),
    }[obj_index]

mp_objectron = mp.solutions.objectron

fx, fy = (1.0, 1.0)
px, py = (0.0, 0.0)
iterations = [1, 2, 5, 6]
h = np.load('h_from_board.npy')


axis_world = np.float32([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])

t_end = time.time() + 60 * 15
while time.time() < t_end:
    _, img = vid.read()
    objectron = mp_objectron.Objectron(mp_objectron.Objectron(static_image_mode=True, max_num_objects=5, min_detection_confidence=0.5, model_name='Shoe'))
    results = objectron.process(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    ann_img = img.copy()
    #warp_img = img.copy()
    center = []
    z_axis = []
    if not results.detected_objects:
        print('no detected objects', end='\r')
        continue
    for obj in results.detected_objects:
        
        x = (int(obj.landmarks_2d.landmark[0].x * img.shape[1]))
        y = (int(obj.landmarks_2d.landmark[0].y * img.shape[0]))
        center.append((x,y))

        axis_cam = np.matmul(obj.rotation, 0.1 * axis_world.T).T + obj.translation
        x_ori = axis_cam[..., 0]
        y_ori = axis_cam[..., 1]
        z_ori = axis_cam[..., 2]
    # Project 3D points to NDC space.
        x_ndc = np.clip(-fx * x_ori / (z_ori + 1e-5) + px, -1., 1.)
        y_ndc = np.clip(-fy * y_ori / (z_ori + 1e-5) + py, -1., 1.)
    # Convert from NDC space to image space.
        x_im = np.int32((1 + x_ndc) * 0.5 * img.shape[1])
        y_im = np.int32((1 - y_ndc) * 0.5 * img.shape[0])
        z_axis.append((x_im[3], y_im[3]))

    comb = combinations(center, 2)
    comb = list(comb)
    dist = []
    try:
        for i in range(len(comb)):

            p1, p2 = comb[i]
            dist.append(math.dist(p1, p2))

        sorted = dist.copy()
        sorted.sort()
        obj_index = dist.index(sorted[0]), dist.index(sorted[1])
        pairs = fetch_value(obj_index[0]), fetch_value(obj_index[1]), fetch_value(obj_index[2])
        #second_pair = fetch_value(obj_index[1])
        for pair in pairs:
            x, y = midpoint(center, pair)
            z0, z1 = midpoint(z_axis, pair)
            cv2.circle(ann_img, (int(x), int(y)), radius=60, color=(0, 0, 255), thickness=5)
            cv2.arrowedLine(ann_img, (int(x), int(y)), (int(z0), int(z1)), color=(0, 128, 0), thickness=3, tipLength=3)
        cv2.imshow('img', ann_img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    
    except:
        print('not enough points', end='\r')

cv2.waitKey(0)
cv2.destroyAllWindows()
vid.release()